 # Hacktober NLP Project

## Goals:
  - Identify the best predictors for programming language
  - Build models to best predict the language used

## Plan of Action:
  - Acquire
  - Prepare
  - Explore
  - Model
  - Conclusions
  

## Data Dictionary

| Target Variable | Definition |
| ----- | ----- |
| language | The main programming language used|

| Feature | Definition | Data Type |
| ----- | ----- | ----- |
| repo | The repository obtained from Github| `object` |
| language| The main programming language used | `object` |
| readme_content | The contents included in the ReadMe file | `object` |
| lem | Readme contents that have been lemmatized | `object` |

In [1]:
import pandas as pd
import numpy as np
import time
import os
from typing import Dict, List, Optional, Union, cast
from bs4 import BeautifulSoup
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

## Acquire:
   - Web scrape data from Github
   - Download it as a json file

## Prepare:
  - Use regex to clean our readme contents
  - Lemmatize the readme contents
  - Replace null values with the correct programming language
  - Split data into Train, Validate, Test

## Modeling
  - Used the found predictors from explore to make models
  - The best model was [Insert Model]

## Conclusion:
   - The